In [1]:
import numpy as np

In [2]:
from sympy.abc import x
from sympy.utilities.lambdify import lambdify, implemented_function
from sympy import diff

In [3]:
m = 2

In [4]:
R = np.arange(1, m)

In [5]:
R

array([1])

In [6]:
# C = [[round(np.random.rand(), 2) if i != j else 0 for j in range(m)] for i in range(m)]

In [7]:
C = np.array([[i+j if i != j else 0 for j in range(m)] for i in range(m)])

In [8]:
C

array([[0, 1],
       [1, 0]])

In [9]:
assert(C.shape == (m, m))

In [10]:
def rho(S):
    return 1
# rho(S) = const - rate of value discount

In [11]:
# dS = mu(S, R)dt + sigma(S, R)dW, W-standard Brownian motion, t-time unit
# mu(S, R) = dS/dt - sigma(S, R)dW/dt
# sigma(S, R) = dS/dW - mu(S, R)dt/dW
# V(S, R) - value function (returns value for regime R in state S)
# V(S, i) ~ phi(S) @ Theta[i]
# f(S, R) - function of payment per unit time in regime R in state S
# beta(S, R) = rho * V(S, R) - L @ V(S, R)

# Condition to switch to x:
# 0 = min(beta(S, R) - f(S, R), min{foreach x != R}(V(S, R) - (V(S, x) - C[R][x])))
# if V(S, R) = V(S, x) - C[R][x] then switch to x

In [12]:
n = 3

In [13]:
S = [1 for i in range(n)] # S - set of nodal values {s}

In [14]:
# phi = [] - array of approximating functions

In [15]:
phi = [(lambda x: x ** 2) for j in range(n)] # phi = [phi[1], phi[2], ...]

In [16]:
# def generatePhi(S):
#     return [[phi[j](S[i]) for i in range(n)] for j in range(n)]

In [17]:
Phi = np.array([[lambdify(x, phi[i](x))(S[j]) for i in range(n)] for j in range(n)])

In [18]:
assert(Phi.shape == (n, n))

In [19]:
def f(S, R):
    return 1

In [20]:
# def f_i(S, i):
#     return np.array([f(S[j], i) for j in range(n)])

In [21]:
f_i = np.array([[f(S[j], i) for j in range(n)] for i in range(n)])

In [22]:
assert(f_i.shape == (n, n))

In [23]:
# def generateF(S):
#     return [[f(S[i], j) for i in range(n)] for j in range(n)]

In [24]:
def mu(s, i):
    return 1
def sigma(s, i):
    return 1

In [25]:
def beta(s, i):
    rho_val = rho(s)
    mu_val = mu(s, i)
    sigma_val = sigma(s, i)

    phi_x = np.array([lambdify(x, phi[j](x))(s) for j in range(n)])
    phi_dx = np.array([lambdify(x, diff(phi[j](x)))(s) for j in range(n)])
    phi_dx2 = np.array([lambdify(x, diff(phi[j](x), x, 2))(s) for j in range(n)])


    return rho_val * phi_x - mu_val * phi_dx + (sigma_val ** 2) / 2 * phi_dx2

In [26]:
# Condition to stay:
# phi_S = np.array([lambdify(x, phi[i](x))(S) for i in range(n)])
# beta(S, i) @ theta[i] - f(S, i) >= 0
# phi_S @ theta[i] - phi_S @ theta[j] + C[i][j] >= 0 for i != j

In [27]:
# def Beta(i):
#     return np.array([beta(S[j], i) for j in range(n)])

In [28]:
Beta = np.array([[beta(S[j], i) for j in range(n)] for i in range(n)])

In [29]:
assert(Beta[0].shape == (n, n))

In [30]:
# 0 = min(M[0]z + q[0], M[1]z + q[1], ...)

In [31]:
e = np.eye(m)
e

array([[1., 0.],
       [0., 1.]])

In [32]:
# def q(i):
#     C_one = np.array([[C[j][i] * np.ones(n).T for j in range(m)] - (e[i] @ f_i[i]) for i in range(n)])
    
#     return C_one

In [44]:
q = np.array(
    [ 
        np.array([C[j][i] * np.ones(n).T for j in range(m)]).reshape((m*n, 1))
        - np.kron(e[i], f_i[i]).reshape(m*n, 1) 
        for i in range(m)
    ]
)

In [45]:
# mul - ?
# q = np.array([ np.array([C[j][i] * np.ones(n).T for j in range(m)]).reshape((m*n, 1)) - mul(e[i] @ f_i[i]).reshape((m*n, 1)) for i in range(n)])

In [46]:
assert(q[0].shape == (m*n, 1))

In [38]:
# def M(i):
#     return (e[i].T @ e[i]) * Beta[i] + ((np.ones((n,n)) - (np.ones(m).reshape(m, 1) @ e[i].reshape(1, m))) * Phi)

In [40]:
M = np.array(
    [
        np.kron(
            (e[i].reshape(m, 1) @ e[i].reshape(m, 1).T),
            Beta[i]
        )
        + np.kron(
            (np.eye(m) - (np.ones(m).reshape(m, 1) @ e[i].reshape(m, 1).T)),
            Phi
        )
        for i in range(m)
    ]
)

In [42]:
# mul - ?
# M = np.array([mul((e[i].T @ e[i]), Beta[i]) + mul((np.ones((m,m)) - (np.ones(m).reshape(m, 1) @ e[i].reshape(1, m))), Phi) for i in range(n)])

In [43]:
assert(M[0].shape == (m*n, m*n))

In [ ]:
# z = m theta_i (shape n, m) - solution to EVLCP: 0 = min(M_1 @ z + q_1, ...)
# w_i = M_i z + q_i >= 0 && (elementwise product of w_i) = 0